In [1]:
from transformers import pipeline

f:\__ing\llming_1\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models_list = [
    'bert-base-cased','roberta-base','distilroberta-base','distilbert-base-cased'
]

for model in models_list:
    nlp = pipeline(
        task="fill-mask",model=model
    )

    print(type(nlp.model))

    preds = nlp(
        f"If you don't {nlp.tokenizer.mask_token} lying, you will end up as a BBC reporter."
    )

    print(
        f"If you don't *** lying, you will end up as a BBC reporter."
    )

    print(type(preds))


    for p in preds:
        print(
            f"Token:{p['token_str']}. Score: {100*p['score']:,.2f}%"
        )
    
    print('\n\n\n')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>
If you don't *** lying, you will end up as a BBC reporter.
<class 'list'>
Token:stop. Score: 33.34%
Token:keep. Score: 26.61%
Token:start. Score: 9.33%
Token:quit. Score: 6.34%
Token:like. Score: 5.86%




<class 'transformers.models.roberta.modeling_roberta.RobertaForMaskedLM'>
If you don't *** lying, you will end up as a BBC reporter.
<class 'list'>
Token: stop. Score: 75.30%
Token: start. Score: 19.09%
Token: quit. Score: 2.58%
Token: keep. Score: 0.68%
Token: practice. Score: 0.49%






Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<class 'transformers.models.roberta.modeling_roberta.RobertaForMaskedLM'>
If you don't *** lying, you will end up as a BBC reporter.
<class 'list'>
Token: stop. Score: 78.28%
Token: start. Score: 6.65%
Token: keep. Score: 2.80%
Token: admit. Score: 2.49%
Token: accept. Score: 2.04%




<class 'transformers.models.distilbert.modeling_distilbert.DistilBertForMaskedLM'>
If you don't *** lying, you will end up as a BBC reporter.
<class 'list'>
Token:stop. Score: 16.91%
Token:be. Score: 5.68%
Token:mind. Score: 4.38%
Token:start. Score: 4.07%
Token:want. Score: 3.96%






# BERT for Sequence Classification

In [2]:
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizerFast, \
    DataCollatorWithPadding, pipeline

from datasets import load_metric, Dataset
import numpy as np

f:\__ing\llming_1\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('./data/snips.train.txt','rb') as f:
    snips_rows = f.readlines()

snips_rows[:20]

[b'listen O\r\n',
 b'to O\r\n',
 b'westbam B-artist\r\n',
 b'alumb O\r\n',
 b'allergic B-album\r\n',
 b'on O\r\n',
 b'google B-service\r\n',
 b'music I-service\r\n',
 b'PlayMusic\r\n',
 b'\r\n',
 b'add O\r\n',
 b'step B-entity_name\r\n',
 b'to I-entity_name\r\n',
 b'me I-entity_name\r\n',
 b'to O\r\n',
 b'the O\r\n',
 b'50 B-playlist\r\n',
 b'cl\xc3\xa1sicos I-playlist\r\n',
 b'playlist O\r\n',
 b'AddToPlaylist\r\n']

In [4]:
utterances = []
tokenized_utterances = []
labels_for_tokens_orig = []
sequence_labels_orig = []

utterance, tokenized_utterance, label_for_utterances = '', [], []

for i,snip_row in enumerate(snips_rows):
    if snip_row == b'\r\n': continue;

    if ' ' not in snip_row.decode():
        sequence_labels_orig.append(snip_row.decode().strip())
        utterances.append(utterance.strip())
        tokenized_utterances.append(tokenized_utterance)
        labels_for_tokens_orig.append(label_for_utterances)
        utterance = ''
        tokenized_utterance = []
        label_for_utterances = []
        continue

    token, token_label = snip_row.decode().split(' ')
    token_label = token_label.strip()

    utterance += f'{token} '

    tokenized_utterance.append(token)
    label_for_utterances.append(token_label)

In [5]:
# 13084 utternace (command to do an action)
# Each utternace consists of words

# list/utterance of words labels, list/utterance of words, string/utterance, entry/utterance of each sequence label
len(labels_for_tokens_orig), len(tokenized_utterances), len(utterances), len(sequence_labels_orig)

(13084, 13084, 13084, 13084)

In [6]:
print(tokenized_utterances[0])
print(labels_for_tokens_orig[0])
print(utterances[0])
print(sequence_labels_orig[0])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
['O', 'O', 'B-artist', 'O', 'B-album', 'O', 'B-service', 'I-service']
listen to westbam alumb allergic on google music
PlayMusic


In [7]:
from functools import reduce

unique_token_labels = list(
    set(
        reduce(
            lambda x,y:x+y,
            labels_for_tokens_orig
        )
    )
)
labels_for_tokens = [
    [
        unique_token_labels.index(_) for _ in l
    ] for l in labels_for_tokens_orig
]

print(
    "Number of unique token labels:",unique_token_labels.__len__(),end="\n\n"
)

print(
    "Examples:",unique_token_labels[:5]
)

Number of unique token labels: 72

Examples: ['I-timeRange', 'B-object_part_of_series_type', 'B-playlist', 'I-playlist_owner', 'B-artist']


In [8]:
unique_sequence_labels = list(set(sequence_labels_orig))
sequence_labels = [unique_sequence_labels.index(l) for l in sequence_labels_orig]
print("Unique sequence labels:\n\t",unique_sequence_labels,end="",sep="")

Unique sequence labels:
	['SearchScreeningEvent', 'RateBook', 'AddToPlaylist', 'SearchCreativeWork', 'GetWeather', 'PlayMusic', 'BookRestaurant']

In [9]:
print(tokenized_utterances[0])
print(labels_for_tokens[0])
print([ unique_token_labels[l] for l in labels_for_tokens[0] ])
print(utterances[0])
print(sequence_labels[0])
print(unique_sequence_labels[sequence_labels[0]])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
[65, 65, 4, 65, 9, 65, 17, 8]
['O', 'O', 'B-artist', 'O', 'B-album', 'O', 'B-service', 'I-service']
listen to westbam alumb allergic on google music
5
PlayMusic


In [10]:
snips_dataset = Dataset.from_dict(
    dict(
        utterance=utterances,
        label=sequence_labels,
        tokens=tokenized_utterances,
        token_labels=labels_for_tokens
    )
)
snips_dataset = snips_dataset.train_test_split(seed=10,test_size=.2)
snips_dataset['train'][111]

{'utterance': 'play tuomas holopainen s the 21 project',
 'label': 5,
 'tokens': ['play', 'tuomas', 'holopainen', 's', 'the', '21', 'project'],
 'token_labels': [65, 4, 24, 65, 9, 38, 38]}

In [11]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [12]:
def preprocess_function(examples):
    return tokenizer(examples["utterance"], truncation=True)

### Example:
ex = snips_dataset['train'][111]
print(ex)
print(preprocess_function(ex)['input_ids'][1:-1])
print(tokenizer.convert_ids_to_tokens(preprocess_function(ex)['input_ids'][1:-1]))
print(
    tokenizer.decode(
        preprocess_function(ex)['input_ids'],
        skip_special_tokens=True
    )
)

{'utterance': 'play tuomas holopainen s the 21 project', 'label': 5, 'tokens': ['play', 'tuomas', 'holopainen', 's', 'the', '21', 'project'], 'token_labels': [65, 4, 24, 65, 9, 38, 38]}
[2377, 10722, 9626, 2015, 7570, 4135, 4502, 21820, 1055, 1996, 2538, 2622]
['play', 'tu', '##oma', '##s', 'ho', '##lo', '##pa', '##inen', 's', 'the', '21', 'project']
play tuomas holopainen s the 21 project


In [13]:
# Create input_ids & attention_mask by mapping the tokenizer to each example
seq_clf_tokenized_snips = snips_dataset.map(
    preprocess_function, batched=True
)

print()
print("Before:",snips_dataset['train'])
print("After:",seq_clf_tokenized_snips['train'])

print("\nIf Padding was used in tokenizer, all the data points will be padded with zeros till the length of the longest data point")
print([x['input_ids'] for x in seq_clf_tokenized_snips['train']][:5])

Map: 100%|██████████| 2617/2617 [00:00<00:00, 16797.42 examples/s]



Before: Dataset({
    features: ['utterance', 'label', 'tokens', 'token_labels'],
    num_rows: 10467
})
After: Dataset({
    features: ['utterance', 'label', 'tokens', 'token_labels', 'input_ids', 'attention_mask'],
    num_rows: 10467
})

If Padding was used in tokenizer, all the data points will be padded with zeros till the length of the longest data point
[[101, 1999, 2698, 2847, 2013, 2085, 2097, 2009, 4542, 2012, 2026, 2783, 2173, 102], [101, 5587, 2023, 2053, 6844, 4328, 24529, 23049, 2072, 8694, 2000, 2026, 2980, 2160, 2377, 9863, 102], [101, 1045, 3446, 2023, 16432, 1014, 102], [101, 2507, 2033, 1996, 3185, 6134, 2005, 3152, 1999, 1996, 2181, 102], [101, 2338, 1037, 3962, 2005, 2416, 2012, 1037, 4825, 2008, 4240, 3869, 1998, 11772, 102]]


In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Example 1:
data_collator([
    {'input_ids': [101, 2023, 2003, 1037, 24, 102]},
    {'input_ids': [101, 2054, 3185, 102]},
])

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[ 101, 2023, 2003, 1037,   24,  102],
        [ 101, 2054, 3185,  102,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0]])}

In [15]:
sequence_clf_model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels = len(unique_sequence_labels)
)

sequence_clf_model.config.id2label = {i: l for i,l in enumerate(unique_sequence_labels)}
sequence_clf_model.config.id2label

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{0: 'SearchScreeningEvent',
 1: 'RateBook',
 2: 'AddToPlaylist',
 3: 'SearchCreativeWork',
 4: 'GetWeather',
 5: 'PlayMusic',
 6: 'BookRestaurant'}

In [16]:
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

metric

C:\Users\Muham\AppData\Local\Temp\ipykernel_16088\1507082491.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = datasets.load_metric("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

   

In [17]:
epochs = 2

training_args = TrainingArguments(
    output_dir="./snips_clf/results",
    num_train_epochs=epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,

    warmup_steps=len(seq_clf_tokenized_snips['train']) // 5,
    weight_decay= 0.05,

    logging_steps=1,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=sequence_clf_model,
    args=training_args,
    train_dataset=seq_clf_tokenized_snips['train'],
    eval_dataset=seq_clf_tokenized_snips['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

# Check if the model is using CUDA (GPU)
is_using_cuda = trainer.args.device.type == 'cuda'
print(f"Is using CUDA: "+'\033[92m'+str(is_using_cuda)+'\033[0m')

Is using CUDA: True


In [18]:
# Get initial accuracy
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, tokens, utterance. If token_labels, tokens, utterance are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [00:02<00:00, 33.21it/s]


{'eval_loss': 1.9463006258010864,
 'eval_accuracy': 0.16431027894535727,
 'eval_runtime': 4.4855,
 'eval_samples_per_second': 583.439,
 'eval_steps_per_second': 18.281}

In [19]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, tokens, utterance. If token_labels, tokens, utterance are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10,467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656
  Number of trainable parameters = 66,958,855
  0%|          | 3/656 [00:00<01:10,  9.31it/s]

{'loss': 1.9267, 'learning_rate': 2.3889154323936934e-08, 'epoch': 0.0}
{'loss': 1.9457, 'learning_rate': 4.777830864787387e-08, 'epoch': 0.01}


  1%|          | 4/656 [00:00<01:08,  9.48it/s]

{'loss': 1.9709, 'learning_rate': 7.16674629718108e-08, 'epoch': 0.01}
{'loss': 1.9498, 'learning_rate': 9.555661729574773e-08, 'epoch': 0.01}


  1%|          | 6/656 [00:00<01:03, 10.19it/s]

...

                                                 
100%|██████████| 656/656 [01:22<00:00,  9.28it/s]Saving model checkpoint to ./snips_clf/results\checkpoint-656
Configuration saved in ./snips_clf/results\checkpoint-656\config.json


{'eval_loss': 0.060164548456668854, 'eval_accuracy': 0.9847153228888039, 'eval_runtime': 2.5331, 'eval_samples_per_second': 1033.119, 'eval_steps_per_second': 32.371, 'epoch': 2.0}


Model weights saved in ./snips_clf/results\checkpoint-656\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./snips_clf/results\checkpoint-656 (score: 0.060164548456668854).
100%|██████████| 656/656 [01:28<00:00,  7.39it/s]

{'train_runtime': 88.7146, 'train_samples_per_second': 235.97, 'train_steps_per_second': 7.395, 'train_loss': 0.7290181125934441, 'epoch': 2.0}


TrainOutput(global_step=656, training_loss=0.7290181125934441, metrics={'train_runtime': 88.7146, 'train_samples_per_second': 235.97, 'train_steps_per_second': 7.395, 'train_loss': 0.7290181125934441, 'epoch': 2.0})

In [20]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, tokens, utterance. If token_labels, tokens, utterance are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [00:02<00:00, 33.90it/s]


{'eval_loss': 0.060164548456668854,
 'eval_accuracy': 0.9847153228888039,
 'eval_runtime': 2.4882,
 'eval_samples_per_second': 1051.75,
 'eval_steps_per_second': 32.955,
 'epoch': 2.0}

In [29]:
pipe = pipeline(
    task="text-classification",
    model=sequence_clf_model, 
    tokenizer=tokenizer,
    device=0
    )
pipe([
    'Add Morph by Twenty One Pilot to my TOP playlist',
    'Rate A thousand splendid suns 9 out of 10 stars'
    ])

[{'label': 'AddToPlaylist', 'score': 0.9911097288131714},
 {'label': 'RateBook', 'score': 0.9908831715583801}]

In [30]:
# Save the best model to the paths specified previously
trainer.save_model()

Saving model checkpoint to ./snips_clf/results
Configuration saved in ./snips_clf/results\config.json
Model weights saved in ./snips_clf/results\pytorch_model.bin


In [31]:
pipe = pipeline(
    task="text-classification",
    model="./snips_clf/results", 
    tokenizer=tokenizer,
    device=0
    )
pipe([
    'Add Morph by Twenty One Pilot to my TOP playlist',
    'Rate A thousand splendid suns 9 out of 10 stars'
    ])

loading configuration file ./snips_clf/results\config.json
Model config DistilBertConfig {
  "_name_or_path": "./snips_clf/results",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "SearchScreeningEvent",
    "1": "RateBook",
    "2": "AddToPlaylist",
    "3": "SearchCreativeWork",
    "4": "GetWeather",
    "5": "PlayMusic",
    "6": "BookRestaurant"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transfor

[{'label': 'AddToPlaylist', 'score': 0.9911097288131714},
 {'label': 'RateBook', 'score': 0.9908831715583801}]

In [32]:
## Freezing some layers
frozen_sequence_clf_model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels = len(unique_sequence_labels)
)

for param in frozen_sequence_clf_model.distilbert.parameters():
    param.requires_grad = False

loading configuration file config.json from cache at C:\Users\Muham/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.35.0",
  "vocab_size": 30522
}


In [64]:
epochs = 2

training_args = TrainingArguments(
    output_dir="./snips_clf/results",
    num_train_epochs=epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,

    warmup_steps=len(seq_clf_tokenized_snips['train']) // 5,
    weight_decay= 0.05,

    logging_steps=1,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=frozen_sequence_clf_model,
    args=training_args,
    train_dataset=seq_clf_tokenized_snips['train'],
    eval_dataset=seq_clf_tokenized_snips['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

# Check if the model is using CUDA (GPU)
is_using_cuda = trainer.args.device.type == 'cuda'
print(f"Is using CUDA: "+'\033[92m'+str(is_using_cuda)+'\033[0m')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Is using CUDA: True


In [65]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, tokens, utterance. If token_labels, tokens, utterance are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [00:02<00:00, 32.17it/s]


{'eval_loss': 1.9450441598892212,
 'eval_accuracy': 0.1700420328620558,
 'eval_runtime': 2.6976,
 'eval_samples_per_second': 970.13,
 'eval_steps_per_second': 30.398}

In [66]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, tokens, utterance. If token_labels, tokens, utterance are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10,467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656
  Number of trainable parameters = 595,975
  1%|          | 6/656 [00:00<00:22, 29.08it/s]

{'loss': 1.953, 'learning_rate': 2.3889154323936934e-08, 'epoch': 0.0}
{'loss': 1.9445, 'learning_rate': 4.777830864787387e-08, 'epoch': 0.01}
{'loss': 1.946, 'learning_rate': 7.16674629718108e-08, 'epoch': 0.01}
{'loss': 1.9548, 'learning_rate': 9.555661729574773e-08, 'epoch': 0.01}
{'loss': 1.9551, 'learning_rate': 1.1944577161968468e-07, 'epoch': 0.02}
{'loss': 1.9433, 'learning_rate': 1.433349259436216e-07, 'epoch': 0.02}


  2%|▏         | 12/656 [00:00<00:22, 28.09it/s]

{'loss': 1.9409, 'learning_rate': 1.6722408026755853e-07, 'epoch': 0.02}
{'loss': 1.9594, 'learning_rate': 1.9111323459149547e-07, 'epoch': 0.02}
{'loss': 1.9559, 'learning_rate': 2.150023889154324e-07, 'epoch': 0.03}
{'loss': 1.9397, 'learning_rate': 2.3889154323936937e-07, 'epoch': 0.03}
{'loss': 1.9657, 'learning_rate': 2.6278069756330625e-07, 'epoch': 0.03}


  3%|▎         | 17/656 [00:00<00:23, 27.60it/s]

...

                                                 
100%|██████████| 656/656 [00:43<00:00, 18.49it/s]Saving model checkpoint to ./snips_clf/results\checkpoint-656
Configuration saved in ./snips_clf/results\checkpoint-656\config.json


{'eval_loss': 1.570827603340149, 'eval_accuracy': 0.8868933893771495, 'eval_runtime': 3.9241, 'eval_samples_per_second': 666.898, 'eval_steps_per_second': 20.896, 'epoch': 2.0}


Model weights saved in ./snips_clf/results\checkpoint-656\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./snips_clf/results\checkpoint-656 (score: 1.570827603340149).
100%|██████████| 656/656 [00:45<00:00, 14.43it/s]

{'train_runtime': 45.4625, 'train_samples_per_second': 460.467, 'train_steps_per_second': 14.429, 'train_loss': 1.8436544395801497, 'epoch': 2.0}


TrainOutput(global_step=656, training_loss=1.8436544395801497, metrics={'train_runtime': 45.4625, 'train_samples_per_second': 460.467, 'train_steps_per_second': 14.429, 'train_loss': 1.8436544395801497, 'epoch': 2.0})

In [67]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, tokens, utterance. If token_labels, tokens, utterance are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [00:02<00:00, 32.01it/s]


{'eval_loss': 1.570827603340149,
 'eval_accuracy': 0.8868933893771495,
 'eval_runtime': 2.7608,
 'eval_samples_per_second': 947.897,
 'eval_steps_per_second': 29.701,
 'epoch': 2.0}

# BERT for Token Classification

In [68]:
from transformers import DataCollatorForTokenClassification, DistilBertForTokenClassification, \
                            DistilBertTokenizerFast, pipeline

In [69]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

loading file vocab.txt from cache at C:\Users\Muham/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\vocab.txt
loading file tokenizer.json from cache at C:\Users\Muham/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\Muham/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\Muham/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_d

In [95]:
snips_dataset['train'][0:2]

# tokenizer(
#         snips_dataset['train'][0:14]['tokens'], truncation=True, is_split_into_words=True
#     ).word_ids(batch_index=13)

{'utterance': ['in seven hours from now will it rain at my current place',
  'add this nozomi tsuji tune to my hot house playlist'],
 'label': [4, 2],
 'tokens': [['in',
   'seven',
   'hours',
   'from',
   'now',
   'will',
   'it',
   'rain',
   'at',
   'my',
   'current',
   'place'],
  ['add',
   'this',
   'nozomi',
   'tsuji',
   'tune',
   'to',
   'my',
   'hot',
   'house',
   'playlist']],
 'token_labels': [[65, 57, 0, 0, 0, 65, 65, 13, 65, 65, 12, 60],
  [65, 65, 4, 24, 47, 65, 36, 2, 23, 65]]}

In [101]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'], truncation=True, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["token_labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100) # Magic number, where the loss is ignored for that label
            elif word_ids != previous_word_idx: # only label the first token of a given word
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Example:
tokenize_and_align_labels(snips_dataset['train'][0:5])

{'input_ids': [[101, 1999, 2698, 2847, 2013, 2085, 2097, 2009, 4542, 2012, 2026, 2783, 2173, 102], [101, 5587, 2023, 2053, 6844, 4328, 24529, 23049, 2072, 8694, 2000, 2026, 2980, 2160, 2377, 9863, 102], [101, 1045, 3446, 2023, 16432, 1014, 102], [101, 2507, 2033, 1996, 3185, 6134, 2005, 3152, 1999, 1996, 2181, 102], [101, 2338, 1037, 3962, 2005, 2416, 2012, 1037, 4825, 2008, 4240, 3869, 1998, 11772, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 65, 57, 0, 0, 0, 65, 65, 13, 65, 65, 12, 60, -100], [-100, 65, 65, 4, 4, 4, 24, 24, 24, 47, 65, 36, 2, 23, 65, 65, -100], [-100, 65, 65, 32, 51, 31, -100], [-100, 65, 65, 65, 51, 45, 65, 69, 44, 18, 18, -100], [-100, 65, 65, 65, 65, 27, 65, 65, 19, 65, 65, 26, 50, 50, -100]]}

In [104]:
tok_clf_tokenized_snips = snips_dataset.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 2617/2617 [00:00<00:00, 7309.02 examples/s]


In [110]:
# Some columns may cause confusion to the token classification model like: label (which is the seq label)
#        so better remove them.
tok_clf_tokenized_snips = tok_clf_tokenized_snips.remove_columns(
    ['utterance', 'label', 'tokens', 'token_labels']
)

In [118]:
[tok_clf_tokenized_snips['train'][0]]

[{'input_ids': [101,
   1999,
   2698,
   2847,
   2013,
   2085,
   2097,
   2009,
   4542,
   2012,
   2026,
   2783,
   2173,
   102],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [-100, 65, 57, 0, 0, 0, 65, 65, 13, 65, 65, 12, 60, -100]}]

In [123]:
tok_data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Example
tok_data_collator([tok_clf_tokenized_snips['train'][0],tok_clf_tokenized_snips['train'][1]])

{'input_ids': tensor([[  101,  1999,  2698,  2847,  2013,  2085,  2097,  2009,  4542,  2012,
          2026,  2783,  2173,   102,     0,     0,     0],
        [  101,  5587,  2023,  2053,  6844,  4328, 24529, 23049,  2072,  8694,
          2000,  2026,  2980,  2160,  2377,  9863,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[-100,   65,   57,    0,    0,    0,   65,   65,   13,   65,   65,   12,
           60, -100, -100, -100, -100],
        [-100,   65,   65,    4,    4,    4,   24,   24,   24,   47,   65,   36,
            2,   23,   65,   65, -100]])}

In [111]:
tok_clf_model = DistilBertForTokenClassification.from_pretrained(
    'distilbert-base-uncased', num_labels=len(unique_token_labels)
)

tok_clf_model.config.id2label = {i: l for i, l in enumerate(unique_token_labels)}
tok_clf_model.config.id2label

loading configuration file config.json from cache at C:\Users\Muham/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": 

{0: 'I-timeRange',
 1: 'B-object_part_of_series_type',
 2: 'B-playlist',
 3: 'I-playlist_owner',
 4: 'B-artist',
 5: 'B-genre',
 6: 'I-cuisine',
 7: 'B-geographic_poi',
 8: 'I-service',
 9: 'B-album',
 10: 'B-track',
 11: 'I-movie_type',
 12: 'B-current_location',
 13: 'B-condition_description',
 14: 'I-restaurant_type',
 15: 'I-genre',
 16: 'B-poi',
 17: 'B-service',
 18: 'I-spatial_relation',
 19: 'B-restaurant_type',
 20: 'I-object_select',
 21: 'I-entity_name',
 22: 'B-restaurant_name',
 23: 'I-playlist',
 24: 'I-artist',
 25: 'B-object_location_type',
 26: 'B-served_dish',
 27: 'B-party_size_number',
 28: 'I-object_location_type',
 29: 'I-city',
 30: 'I-location_name',
 31: 'B-rating_value',
 32: 'B-object_select',
 33: 'I-track',
 34: 'B-movie_name',
 35: 'I-poi',
 36: 'B-playlist_owner',
 37: 'I-state',
 38: 'I-album',
 39: 'B-cuisine',
 40: 'B-year',
 41: 'I-movie_name',
 42: 'B-sort',
 43: 'B-rating_unit',
 44: 'B-spatial_relation',
 45: 'I-object_type',
 46: 'B-city',
 47: 'B

In [124]:
epochs = 2

training_args = TrainingArguments(
    output_dir="./snips_tok_clf/results",
    num_train_epochs=epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,

    logging_steps=10,
    logging_strategy='steps',
    log_level='info',

    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=tok_clf_model,
    args=training_args,
    train_dataset=tok_clf_tokenized_snips['train'],
    eval_dataset=tok_clf_tokenized_snips['test'],
    data_collator=tok_data_collator
)

# Check if the model is using CUDA (GPU)
is_using_cuda = trainer.args.device.type == 'cuda'
print(f"Is using CUDA: "+'\033[92m'+str(is_using_cuda)+'\033[0m')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Is using CUDA: True


In [125]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [00:02<00:00, 31.41it/s]


{'eval_loss': 4.430819034576416,
 'eval_runtime': 2.6395,
 'eval_samples_per_second': 991.46,
 'eval_steps_per_second': 31.066}

In [126]:
trainer.train()

***** Running training *****
  Num examples = 10,467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656
  Number of trainable parameters = 66,418,248
  2%|▏         | 12/656 [00:01<01:07,  9.49it/s]

{'loss': 3.4594, 'learning_rate': 4.923780487804878e-05, 'epoch': 0.03}


  3%|▎         | 21/656 [00:02<01:06,  9.51it/s]

{'loss': 2.672, 'learning_rate': 4.847560975609756e-05, 'epoch': 0.06}


  5%|▍         | 31/656 [00:03<01:08,  9.17it/s]

...

100%|█████████▉| 655/656 [01:19<00:00,  9.11it/s]***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
                                                 
100%|██████████| 656/656 [01:22<00:00,  9.11it/s]Saving model checkpoint to ./snips_tok_clf/results\checkpoint-656
Configuration saved in ./snips_tok_clf/results\checkpoint-656\config.json


{'eval_loss': 0.1553291380405426, 'eval_runtime': 2.5425, 'eval_samples_per_second': 1029.319, 'eval_steps_per_second': 32.252, 'epoch': 2.0}


Model weights saved in ./snips_tok_clf/results\checkpoint-656\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./snips_tok_clf/results\checkpoint-656 (score: 0.1553291380405426).
100%|██████████| 656/656 [01:27<00:00,  7.50it/s]

{'train_runtime': 87.5014, 'train_samples_per_second': 239.242, 'train_steps_per_second': 7.497, 'train_loss': 0.47977145942972926, 'epoch': 2.0}


TrainOutput(global_step=656, training_loss=0.47977145942972926, metrics={'train_runtime': 87.5014, 'train_samples_per_second': 239.242, 'train_steps_per_second': 7.497, 'train_loss': 0.47977145942972926, 'epoch': 2.0})

In [133]:
pipe = pipeline(
    task="token-classification", model=tok_clf_model, tokenizer=tokenizer, device=0
)
pipe([
    'Add Morph by the band Twenty One Pilot to my TOP playlist',
    'Rate A thousand splendid suns 9 out of 10 stars',
    ])

[[{'entity': 'B-entity_name',
   'score': 0.71627563,
   'index': 2,
   'word': 'mor',
   'start': 4,
   'end': 7},
  {'entity': 'B-entity_name',
   'score': 0.6723716,
   'index': 3,
   'word': '##ph',
   'start': 7,
   'end': 9},
  {'entity': 'B-entity_name',
   'score': 0.8071509,
   'index': 7,
   'word': 'twenty',
   'start': 22,
   'end': 28},
  {'entity': 'I-entity_name',
   'score': 0.85959774,
   'index': 8,
   'word': 'one',
   'start': 29,
   'end': 32},
  {'entity': 'I-entity_name',
   'score': 0.88719934,
   'index': 9,
   'word': 'pilot',
   'start': 33,
   'end': 38},
  {'entity': 'B-playlist_owner',
   'score': 0.98588145,
   'index': 11,
   'word': 'my',
   'start': 42,
   'end': 44},
  {'entity': 'B-playlist',
   'score': 0.9719445,
   'index': 12,
   'word': 'top',
   'start': 45,
   'end': 48}],
 [{'entity': 'B-object_name',
   'score': 0.9828287,
   'index': 2,
   'word': 'a',
   'start': 5,
   'end': 6},
  {'entity': 'I-object_name',
   'score': 0.97984385,
   'in

# BERT for question/answering

In [1]:
import torch

# Check if CUDA is available
def show_cuda_space_info():
    if torch.cuda.is_available():
        # Get the CUDA device name
        device = torch.device("cuda")
        print("Using device:", torch.cuda.get_device_name(device))

        # Memory allocation and caching are dynamic in PyTorch, but you can get approximate memory usage as follows
        total_memory = torch.cuda.get_device_properties(device).total_memory
        allocated_memory = torch.cuda.memory_allocated(device)
        cached_memory = torch.cuda.memory_reserved(device)
        free_memory = total_memory - (allocated_memory + cached_memory)

        print(f"Total memory: {total_memory / 1e9:.2f} GB")
        print(f"Allocated memory: {allocated_memory / 1e9:.2f} GB")
        print(f"Cached memory: {cached_memory / 1e9:.2f} GB")
        print(f"Free memory: {free_memory / 1e9:.2f} GB")
    else:
        print("CUDA is not available.")
show_cuda_space_info()

Using device: NVIDIA GeForce GTX 1660 Ti
Total memory: 6.44 GB
Allocated memory: 0.00 GB
Cached memory: 0.00 GB
Free memory: 6.44 GB


In [2]:
from transformers import BertTokenizerFast, BertForQuestionAnswering, pipeline, \
                            DataCollatorWithPadding, TrainingArguments, Trainer, \
                                AutoModelForQuestionAnswering, AutoTokenizer

from datasets import Dataset
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests

f:\__ing\llming_1\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', return_token_type_ids=True)
qa_bert = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
qa_df = pd.read_csv('./data/qa.csv')
qa_df.head()

,question,context,start_positions,end_positions,answer
0,What sare the benifts of the blood brain barrir?,Another approach to brain function is to exami...,56,60,isolated from the bloodstream
1,What is surrounded by cerebrospinal fluid?,Another approach to brain function is to exami...,16,16,brain
2,What does the skull protect?,Another approach to brain function is to exami...,11,11,brain
3,What has been injected into rats to produce pr...,Another approach to brain function is to exami...,153,153,chemicals
4,What can cause issues with how the brain works?,Another approach to brain function is to exami...,93,94,brain damage


In [5]:
# Example:
bert_tokenizer.decode(
    bert_tokenizer.encode(
        qa_df.iloc[0].question,qa_df.iloc[0].context
    )[56:61]
)

'isolated from the bloodstream'

In [6]:
qa_dataset = Dataset.from_pandas(
    qa_df.sample(2000,random_state=100)
).train_test_split(test_size=.2)
qa_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'start_positions', 'end_positions', 'answer', '__index_level_0__'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['question', 'context', 'start_positions', 'end_positions', 'answer', '__index_level_0__'],
        num_rows: 400
    })
})

In [7]:
def preprocess(data):
    return bert_tokenizer(
        text=data['question'], text_pair=data['context'], truncation=True
    )

qa_dataset = qa_dataset.map(preprocess, batched=True)
qa_dataset

Map: 100%|██████████| 400/400 [00:00<00:00, 1746.73 examples/s]


DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'start_positions', 'end_positions', 'answer', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['question', 'context', 'start_positions', 'end_positions', 'answer', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 400
    })
})

In [8]:
show_cuda_space_info()

Using device: NVIDIA GeForce GTX 1660 Ti
Total memory: 6.44 GB
Allocated memory: 0.00 GB
Cached memory: 0.00 GB
Free memory: 6.44 GB


In [12]:
# Freeze all but last 2 layers
for name, param in qa_bert.bert.named_parameters():
    if 'encoder.layer.7' in name:
        break
    param.requires_grad = False

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=bert_tokenizer)

In [14]:
batch_size = 32
epochs = 2

training_args = TrainingArguments(
    output_dir='./qa/results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_dir='./qa/logs',
    save_strategy='epoch',
    logging_steps=10,
    evaluation_strategy='epoch',
    load_best_model_at_end=True
)

trainer = Trainer(
    model=qa_bert,
    args=training_args,
    train_dataset=qa_dataset['train'],
    eval_dataset=qa_dataset['test'],
    data_collator=data_collator
)

# Check if the model is using CUDA (GPU)
is_using_cuda = trainer.args.device.type == 'cuda'
print(f"Is using CUDA: "+'\033[92m'+str(is_using_cuda)+'\033[0m'+'\n\n\n\n')



trainer.evaluate()
show_cuda_space_info()

Is using CUDA: True






100%|██████████| 13/13 [00:12<00:00,  1.03it/s]

Using device: NVIDIA GeForce GTX 1660 Ti
Total memory: 6.44 GB
Allocated memory: 0.45 GB
Cached memory: 2.60 GB
Free memory: 3.40 GB


In [19]:
torch.cuda.empty_cache()
show_cuda_space_info()

Using device: NVIDIA GeForce GTX 1660 Ti
Total memory: 6.44 GB
Allocated memory: 5.30 GB
Cached memory: 5.50 GB
Free memory: -4.36 GB


In [20]:
# trainer.train()
# trainer.save_model()

In [47]:
PERSON = 'Yahya Sinwar'

google_html = BeautifulSoup(
    requests.get(f"https://www.google.com/search?hl=en&q={PERSON}").text
).get_text().split(f"_".join(PERSON.split()))[-1][:512]


google_html

"Yahya Sinwar also spelled Yehya Sinwar, is a Palestinian politician who has been leader of Hamas, the Sunni Islamist political and military organization\xa0...Khan Yunis refugee camp · Essam al-Da'alis · Far'aIsrael finds tunnels under vacation homes of Yahya Sinwar, other ...www.timesofisrael.com › idf-uncovers-tunnels-under-vacation-homes-of-y...1 day ago · Israeli forces announced Sunday that they had discovered tunnels under Gaza vacation homes used by senior Hamas leaders, as well as another\xa0...Gaza's big"

In [48]:
squad_pipe = pipeline(
    "question-answering", "bert-large-uncased-whole-word-masking-finetuned-squad"
)

config.json: 100%|██████████| 443/443 [00:00<00:00, 110kB/s]
f:\__ing\llming_1\.venv\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Muham\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
model.safetensors: 100%|██████████| 1.34G/1.34G [04:51<00:00, 4.60MB/s]
Some weights of t

In [49]:
squad_pipe(f"Who's {PERSON}?", google_html)

{'score': 0.5514828562736511,
 'start': 43,
 'end': 67,
 'answer': 'a Palestinian politician'}

In [55]:
large_tokenizer = AutoTokenizer.from_pretrained(
    'bert-large-uncased-whole-word-masking-finetuned-squad'
)

qa_input = large_tokenizer(
    f"Who's {PERSON}?", google_html, return_tensors='pt'
)

In [56]:
qa_input

{'input_ids': tensor([[  101,  2040,  1005,  1055,  8038, 17915,  8254,  9028,  1029,   102,
          8038, 17915,  8254,  9028,  2036, 11479,  6300, 17915,  8254,  9028,
          1010,  2003,  1037,  9302,  3761,  2040,  2038,  2042,  3003,  1997,
         22129,  1010,  1996, 18883, 27256,  2576,  1998,  2510,  3029,  1012,
          1012,  1012,  4967, 22854,  2483, 13141,  3409,  1087,  9686, 21559,
          2632,  1011,  4830,  1005,  4862,  2015,  1087,  2521,  1005,  9932,
         21338, 21147,  4858, 10633,  2104, 10885,  5014,  1997,  8038, 17915,
          8254,  9028,  1010,  2060,  1012,  1012,  1012,  7479,  1012,  2335,
         11253,  2483, 16652,  2140,  1012,  4012,  1533, 24011,  1011, 26944,
          2015,  1011, 10633,  1011,  2104,  1011, 10885,  1011,  5014,  1011,
          1997,  1011,  1061,  1012,  1012,  1012,  1015,  2154,  3283,  1087,
          5611,  2749,  2623,  4465,  2008,  2027,  2018,  3603, 10633,  2104,
         14474, 10885,  5014,  2109,  

In [57]:
large_qa_bert =  AutoModelForQuestionAnswering.from_pretrained(
    'bert-large-uncased-whole-word-masking-finetuned-squad'
)
output = large_qa_bert(**qa_input)
output

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-4.4032, -6.5777, -5.8264, -7.7264, -7.3004, -9.1275, -9.0435, -9.4126,
         -9.2898, -4.4032,  0.5009, -4.8267, -3.6733, -6.0305, -4.6577, -5.3955,
         -0.8210, -5.2820, -5.6137, -6.2838, -4.6382,  2.5435,  6.7323,  6.2563,
          2.2676, -3.9064, -2.8994, -3.9998,  0.8833, -5.3907, -0.2169, -5.8393,
         -2.9233, -1.4878, -3.0171, -2.4509, -7.0053, -3.5163, -3.5348, -5.1425,
         -6.8108, -7.1112, -2.4771, -6.6830, -7.4937, -6.2401, -7.1022, -5.7704,
         -3.1855, -6.9959, -6.8450, -9.0561, -7.4292, -9.0235, -8.0169, -7.7524,
         -5.5358, -3.0518, -8.7132, -7.2135, -7.6325, -6.8808, -6.7930, -6.5131,
         -8.3057, -7.0755, -7.5506, -8.1282, -4.9998, -8.1438, -7.7102, -8.9431,
         -8.3114, -6.7733, -6.9179, -8.1168, -8.3249, -5.3237, -8.4998, -5.8672,
         -9.1332, -9.3890, -9.2773, -9.2029, -8.7893, -7.8259, -5.1415, -6.3822,
         -7.7428, -7.5734, -8.8970, -8.1382, -7.1561, -8